In [5]:
import os
# train_pos_path = 'HW1_data/training/pos'
# train_pos_files = os.listdir(train_pos_path)
# train_pos = []
# for file in train_pos_files:
#     if not os.path.isdir(train_pos_path + "/" + file):
#         f = open(train_pos_path + "/" + file, encoding='ISO-8859–1')
#         line = f.readline()
#         train_pos.append(line)
#         f.close()

def load_data(path):
    sentence_files = os.listdir(path)
    sentences = []
    for file in sentence_files:
        if not os.path.isdir(path + "/" + file):
            f = open(path + "/" + file, encoding='ISO-8859–1')
            line = f.readline()
            sentences.append(line)
            f.close()
    return sentences
train_pos = load_data('HW1_data/training/pos')
train_neg = load_data('HW1_data/training/neg')

In [ ]:
import nltk
from tqdm import tqdm
# nltk.download('punkt')
# train_pos_dict = {}
# tokenizer = nltk.RegexpTokenizer(r'[a-zA-Z]+')
# for sentence in tqdm(train_pos):
#     tokens = tokenizer.tokenize(sentence)
#     for token in tokens:
#         if token in train_pos_dict.keys():
#             train_pos_dict[token] += 1
#         else:
#             train_pos_dict[token] = 1

# V_pos = len(train_pos_dict.keys())
# Count_pos = sum(train_pos_dict.values())
# train_pos_prob = {}
# for token in train_pos_dict.keys():
#     train_pos_prob[token] = (train_pos_dict[token] + 1) / (Count_pos + V_pos)

In [7]:
def token_count(sentences):
    count_dict = {}
    tokenizer = nltk.RegexpTokenizer(r'[a-zA-Z]+')
    for sentence in tqdm(sentences):
        tokens = tokenizer.tokenize(sentence)
        for token in tokens:
            if token in count_dict.keys():
                count_dict[token] += 1
            else:
                count_dict[token] = 1

    V = len(count_dict.keys())
    Count = sum(count_dict.values())
    prob_dict = {}
    for token in count_dict.keys():
        prob_dict[token] = (count_dict[token] + 1) / (Count + V)
    prob_dict['not_matched'] = 1 / (Count + V)
    return prob_dict
train_pos_dict = token_count(train_pos)
train_neg_dict = token_count(train_neg)

100%|██████████| 552/552 [00:00<00:00, 3109.63it/s]


In [8]:
def train(training_path):
    # load and preprocess data
    train_pos = load_data(training_path + '/pos')
    train_neg = load_data(training_path + '/neg')

    # compute prior
    prior = {}
    prior['pos'] = len(train_pos)/(len(train_pos) + len(train_neg))
    prior['neg'] = len(train_neg)/(len(train_pos) + len(train_neg))

    # prior conditional probability
    train_pos_dict = token_count(train_pos)
    train_neg_dict = token_count(train_neg)

    trained_model = []
    trained_model.append(prior)
    trained_model.append(train_pos_dict)
    trained_model.append(train_neg_dict)
    return trained_model

In [12]:
trained_model = train("HW1_data/training")

100%|██████████| 552/552 [00:00<00:00, 3331.26it/s]


In [62]:
test_data_pos = load_data("HW1_data/testing" + '/pos')
test_data_neg = load_data("HW1_data/testing" + '/neg')
test_data = test_data_pos + test_data_neg

In [66]:
from math import log
log(trained_model[0]['neg'])

-0.6949571358051848

In [9]:
from math import log
tokenizer = nltk.RegexpTokenizer(r'[a-zA-Z]+')
pred = []
for sentence in tqdm(test_data):
    pos_prob = log(trained_model[0]['pos'])
    neg_prob = log(trained_model[0]['neg'])
    tokens = tokenizer.tokenize(sentence)
    for token in tokens:
        if token in trained_model[1].keys():
            pos_prob += log(trained_model[1][token])
        else:
            pos_prob += log(trained_model[1]["not_matched"])
        
        if token in trained_model[2].keys():
            neg_prob += log(trained_model[2][token])
        else:
            neg_prob += log(trained_model[2]["not_matched"])
    if pos_prob > neg_prob:
        pred.append(1)
    else:
        pred.append(0)
        

NameError: name 'test_data' is not defined

In [10]:
TESTING_PATH = "HW1_data/testing"
def test(trained_model, testing_path):
    test_data_pos = load_data(TESTING_PATH + '/pos')
    test_data_neg = load_data(TESTING_PATH + '/neg')
    test_data = test_data_pos + test_data_neg
    ground_truth = [1] * len(test_data_neg) + [0]*len(test_data_neg)
    tokenizer = nltk.RegexpTokenizer(r'[a-zA-Z]+')
    model_predictions = []
    for sentence in tqdm(test_data):
        pos_prob = log(trained_model[0]['pos'])
        neg_prob = log(trained_model[0]['neg'])
        tokens = tokenizer.tokenize(sentence)
        for token in tokens:
            if token in trained_model[1].keys():
                pos_prob += log(trained_model[1][token])
            else:
                pos_prob += log(trained_model[1]["not_matched"])
            
            if token in trained_model[2].keys():
                neg_prob += log(trained_model[2][token])
            else:
                neg_prob += log(trained_model[2]["not_matched"])
        if pos_prob > neg_prob:
            model_predictions.append(1)
        else:
            model_predictions.append(0)
    return model_predictions, ground_truth

In [13]:
pred, truth = test(trained_model, TESTING_PATH)

100%|██████████| 140/140 [00:00<00:00, 1332.29it/s]


In [14]:
cum = 0
for i in range(len(pred)):  
    if pred[i]==truth[i]:
        cum += 1
cum / 140

0.8071428571428572